In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd

import time
import os.path

import shapely.geometry as geom
import geopandas as gpd
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely.geometry import Point

In [3]:
link_base = 'link.csv'
TMC_file = 'TMC_Identification_I10.csv'
'''reading link_base'''
link_base = pd.read_csv(link_base, low_memory=False)
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk'])]
# link_base = link_base[-link_base['name'].isna()]
# link_base = link_base[link_base['link_type_name'].isin(['motorway','trunk','primary','secondary'])]
# link_base = link_base.reset_index()
# link_base = link_base.drop(['index'], 1)

'''convert link_base to MultiLineString''' 
multiline_string_base_list = []
multiline_string_base_list_sub = []
for j in link_base.index:
    link_base_geometry_list = link_base.loc[j,'geometry'][12:-1].split(", ")
    for link_base_geometry in link_base_geometry_list:
        multiline_string_base_list_sub.append((float(link_base_geometry.split(" ")[0]),float(link_base_geometry.split(" ")[1])))
    multiline_string_base_list_sub = tuple(multiline_string_base_list_sub)
    multiline_string_base_list.append(multiline_string_base_list_sub)
    multiline_string_base_list_sub = []

from shapely.geometry import MultiLineString
line_base = MultiLineString(multiline_string_base_list) 

'''reading tmc'''
tmc = pd.read_csv(TMC_file)
tmc = tmc.drop_duplicates(subset=['direction','road_order']).sort_values(by=['direction','road_order'])
tmc = tmc.reset_index()
tmc = tmc.drop(['index'], 1)
origin_tmc_num = len(tmc)

'''remove out boundary tmc'''
in_bbox_index_list = []
for i in tmc.index:
    if (tmc['start_longitude'][i] > line_base.bounds[0]) & (tmc['start_longitude'][i] < line_base.bounds[2]) & \
        (tmc['end_longitude'][i] > line_base.bounds[0]) & (tmc['end_longitude'][i] < line_base.bounds[2]) & \
            (tmc['start_latitude'][i] > line_base.bounds[1]) & (tmc['start_latitude'][i] < line_base.bounds[3]) & \
        (tmc['end_latitude'][i] > line_base.bounds[1]) & (tmc['end_latitude'][i] < line_base.bounds[3]):
        in_bbox_index_list.append(i)

tmc = tmc.loc[in_bbox_index_list]
tmc = tmc.reset_index()
tmc = tmc.drop(['index'], 1)
if len(in_bbox_index_list) < origin_tmc_num:
    print('base map cannot cover all TMC nodes,' + str(origin_tmc_num-len(in_bbox_index_list)) + 'tmc nodes are out of boundary box, please use larger base map')




'''build node.csv'''
print('converting tmc data into gmns format...')
p=1

node_tmc = pd.DataFrame()
node_tmc['name'] = None
node_tmc['x_coord'] = None
node_tmc['y_coord'] = None
node_tmc['z_coord'] = None
node_tmc['node_type'] = None
node_tmc['ctrl_type'] = None
node_tmc['zone_id'] = None
node_tmc['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' nodes completed!')
        p = p + 1

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc.index.name = 'node_id'

node_tmc.index += 100000001 #index from 0

node_tmc.to_csv('node_tmc.csv')
print('node_tmc.csv (' + str(len(node_tmc)) + ' nodes' + ') generated!')


'''build link_tmc.csv'''
p = 1
link_tmc = pd.DataFrame()
link_tmc['name'] = None
link_tmc['corridor_id'] = None
link_tmc['corridor_link_order'] = None
link_tmc['from_node_id'] = None
link_tmc['to_node_id'] = None
link_tmc['directed'] = None
link_tmc['geometry_id'] = None
link_tmc['geometry'] = None
link_tmc['dir_flag'] = None
link_tmc['length'] = None
link_tmc['grade'] = None
link_tmc['facility_type'] = None
link_tmc['capacity'] = None
link_tmc['free_speed'] = None
link_tmc['lanes'] = None

for i in range(0,len(tmc)):
    link_tmc = link_tmc.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                'from_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)

    if i > p/10 * len(tmc): 
        print(str(p*10)+"%"+' links completed!')
        p = p + 1
        
link_tmc.index.name = 'link_id'
link_tmc.index += 100000001

link_tmc.to_csv('link_tmc.csv')

print('link_tmc.csv (' + str(len(link_tmc)) + ' links' + ') generated!')

converting tmc data into gmns format...
10% nodes completed!
20% nodes completed!
30% nodes completed!
40% nodes completed!
50% nodes completed!
60% nodes completed!
70% nodes completed!
80% nodes completed!
90% nodes completed!
node_tmc.csv (71 nodes) generated!


IndexError: index 0 is out of bounds for axis 0 with size 0

In [21]:
tmc

,tmc,road,direction,intersection,state,county,zip,start_latitude,start_longitude,end_latitude,end_longitude,miles,road_order,timezone_name,type,country,active_start_date,active_end_date
0,115P08698,QUEEN CREEK RD,EASTBOUND,I-10,AZ,MARICOPA,85226,33.25611,-111.94709,33.25617,-111.94698,0.007587,1.0,America/Phoenix,P1.3,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
1,115+08699,QUEEN CREEK RD,EASTBOUND,S PRICE RD,AZ,MARICOPA,85226,33.25617,-111.94698,33.26141,-111.88886,3.432512,2.0,America/Phoenix,P1.11,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
2,115+08700,QUEEN CREEK RD,EASTBOUND,S DOBSON RD,AZ,MARICOPA,85248,33.26141,-111.88868,33.26153,-111.88114,0.435621,4.0,America/Phoenix,P1.11,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
3,115+08701,QUEEN CREEK RD,EASTBOUND,S ALMA SCHOOL RD,AZ,MARICOPA,85248,33.26153,-111.88095,33.26185,-111.85868,1.286592,6.0,America/Phoenix,P1.11,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
4,115+08702,QUEEN CREEK RD,EASTBOUND,AZ-87/S ARIZONA AVE,AZ,MARICOPA,85248,33.26185,-111.85850,33.26206,-111.84143,0.986150,8.0,America/Phoenix,P1.11,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,115+04179,I-10,WESTBOUND,WARNER RD/EXIT 158,AZ,MARICOPA,85226,33.32390,-111.97209,33.33027,-111.97208,0.440047,1115.0,America/Phoenix,P1.3,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
63,115P04179,I-10,WESTBOUND,WARNER RD/EXIT 158,AZ,MARICOPA,85284,33.33027,-111.97208,33.34022,-111.97208,0.687359,1116.0,America/Phoenix,P1.3,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
64,115+04180,I-10,WESTBOUND,ELLIOT RD/EXIT 157,AZ,MARICOPA,85284,33.34022,-111.97208,33.34452,-111.97209,0.297069,1117.0,America/Phoenix,P1.3,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00
65,115P04180,I-10,WESTBOUND,ELLIOT RD/EXIT 157,AZ,MARICOPA,85283,33.34452,-111.97209,33.35384,-111.97098,0.647633,1118.0,America/Phoenix,P1.3,USA,2018-12-03 12:00:00-05:00,2019-04-16 17:00:00-04:00


In [18]:
i=3

In [19]:
node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0]

100000004

In [20]:
node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,length,grade,facility_type,capacity,free_speed,lanes
0,115P08698,QUEEN CREEK RD_EASTBOUND,1.0,100000001,100000002,1,None,"LINESTRING (-111.94708999999999 33.25611,-111....",1,0.007587,None,None,None,None,None


In [2]:
import network_matching_function as nmf 
nmf.TMCIdentification2GMNSNodeLinkFiles('TMC_Identification_I10.csv','link.csv')

converting tmc data into gmns format...
10% nodes completed!
20% nodes completed!
30% nodes completed!
40% nodes completed!
50% nodes completed!
60% nodes completed!
70% nodes completed!
80% nodes completed!
90% nodes completed!
node_tmc.csv (71 nodes) generated!


IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
nmf.ConvertTMCReading2Measurement('Reading.csv','link_tmc.csv')


10% measurement_tmc completed!
20% measurement_tmc completed!
30% measurement_tmc completed!
40% measurement_tmc completed!
50% measurement_tmc completed!
60% measurement_tmc completed!
70% measurement_tmc completed!
80% measurement_tmc completed!
90% measurement_tmc completed!
measurement_tmc.csv generated!


In [5]:
nmf.MatchTMC2GMNSNetwork('link_tmc.csv','link.csv')

10% matching completed!
20% matching completed!
30% matching completed!
40% matching completed!
50% matching completed!
60% matching completed!
70% matching completed!
80% matching completed!
90% matching completed!
matching_tmc2gmns.csv generated!


In [6]:
nmf.ConvertMeasurementBasedOnMatching('link.csv','matching_tmc2gmns.csv','measurement_tmc.csv')

10% measurement_base completed!
20% measurement_base completed!
30% measurement_base completed!
40% measurement_base completed!
50% measurement_base completed!
60% measurement_base completed!
70% measurement_base completed!
80% measurement_base completed!
90% measurement_base completed!
100% measurement_base completed!
measurement_base.csv generated!


In [5]:
#intergrate all functions in one

import network_matching_function as nmf 
nmf.OneFunction()

converting tmc data into gmns format...
10% nodes completed!
20% nodes completed!
30% nodes completed!
40% nodes completed!
50% nodes completed!
60% nodes completed!
70% nodes completed!
80% nodes completed!
90% nodes completed!
node_tmc.csv (149 nodes) generated!
10% links completed!
20% links completed!
30% links completed!
40% links completed!
50% links completed!
60% links completed!
70% links completed!
80% links completed!
90% links completed!
link_tmc.csv (144 links) generated!
10% measurement_tmc completed!
20% measurement_tmc completed!
30% measurement_tmc completed!
40% measurement_tmc completed!
50% measurement_tmc completed!
60% measurement_tmc completed!
70% measurement_tmc completed!
80% measurement_tmc completed!
90% measurement_tmc completed!
measurement_tmc.csv generated!
10% initial matching completed!
20% initial matching completed!
30% initial matching completed!
40% initial matching completed!
50% initial matching completed!
60% initial matching completed!
70% init